In [1]:
!pip install numpy opencv-python pillow tqdm requests moviepy

In [8]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import os
import random
import logging
from google.colab import auth
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from google.colab import files

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# YouTube API scopes
SCOPES = ['https://www.googleapis.com/auth/youtube.upload']

class YouTubeShortsUploader:
    def __init__(self):
        # Authenticate and create YouTube API client
        self.youtube = self._get_authenticated_service()

    def _get_authenticated_service(self):
        """
        Authenticate and create a YouTube API service for Colab.
        """
        # Authenticate via Google Colab
        auth.authenticate_user()

        # Attempt to load existing credentials
        token_file = 'youtube_upload_token.json'
        credentials = None

        try:
            if os.path.exists(token_file):
                credentials = Credentials.from_authorized_user_file(token_file, SCOPES)

            # Refresh token if expired
            if credentials and credentials.expired and credentials.refresh_token:
                credentials.refresh(Request())

            # If no valid credentials, initiate OAuth flow
            if not credentials or not credentials.valid:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'client_secrets.json', SCOPES)
                credentials = flow.run_local_server(port=0)

                # Save credentials for future use
                with open(token_file, 'w') as token:
                    token.write(credentials.to_json())

        except Exception as e:
            logging.error(f"Authentication error: {e}")
            # Prompt user to upload client_secrets.json if not found
            logging.info("Please upload client_secrets.json file.")
            uploaded_files = files.upload()
            if 'client_secrets.json' not in uploaded_files:
                raise ValueError("client_secrets.json is required for authentication")

            # Retry authentication
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            credentials = flow.run_local_server(port=0)

        return build('youtube', 'v3', credentials=credentials)

    def _generate_dynamic_description(self, book_number):
        """
        Generate an engaging, random description for the video.
        """
        book_genres = [
            "fantasy adventure", "sci-fi thriller", "mystery novel",
            "historical fiction", "romantic comedy", "psychological drama"
        ]

        viral_hooks = [
            "You won't believe what happens next! 🤯",
            "The twist will blow your mind! 😱",
            "A story that changed everything... 🌟",
            "Secret revealed: The untold story! 🔍",
            "Life-changing moments captured! 💥"
        ]

        call_to_actions = [
            "Follow for more epic stories! 🚀",
            "Double tap if this resonates! ❤️",
            "Comment your favorite moment! 💬",
            "Share with a friend who needs this! 🤝",
            "Tag someone who must see this! 👀"
        ]

        genre = random.choice(book_genres)
        hook = random.choice(viral_hooks)
        cta = random.choice(call_to_actions)

        descriptions = [
            f"Dive into an incredible {genre} journey that will transform your perspective! {hook}\n\n"
            f"Book {book_number}: A story of courage, passion, and unexpected twists. 📖✨\n\n"
            f"{cta}\n\n"
            "#BookTok #BookRecommendation #StoryTime #BookLovers #ReadingCommunity",

            f"Unlock a world of imagination with this mind-blowing {genre} narrative! {hook}\n\n"
            f"Experience the magic of Book {book_number} - where reality meets extraordinary! 🌈\n\n"
            f"{cta}\n\n"
            "#StorytimeShorts #BookReview #MustRead #BookNerd #LiteraryWorld",

            f"Prepare for an emotional rollercoaster in this {genre} masterpiece! {hook}\n\n"
            f"Book {book_number}: A journey that will stay with you forever. 💖\n\n"
            f"{cta}\n\n"
            "#BookShorts #StorytellingMagic #EmotionalJourney #BookRecommendations #LoveReading"
        ]

        return random.choice(descriptions)

    def _generate_tags(self, book_number):
        """
        Generate a diverse set of tags to improve discoverability.
        """
        base_tags = [
            f"Book{book_number}", "BookTok", "BookRecommendation",
            "StoryTime", "BookLovers", "BookShorts"
        ]

        genre_tags = [
            "FantasyBook", "SciFiStory", "MysteryNovel",
            "HistoricalFiction", "RomanceBook", "DramaBook"
        ]

        trending_tags = [
            "BookReview", "MustRead", "ReadingCommunity",
            "BookNerd", "LiteraryWorld", "StorytellingMagic"
        ]

        return base_tags + [random.choice(genre_tags)] + random.sample(trending_tags, 3)

    def upload_short(self, video_path, book_number):
        """
        Upload a video as a YouTube Short with dynamic metadata.
        """
        try:
            body = {
                'snippet': {
                    'title': f'Epic Story Journey - Book {book_number}',
                    'description': self._generate_dynamic_description(book_number),
                    'tags': self._generate_tags(book_number),
                    'categoryId': '22'  # Entertainment category
                },
                'status': {
                    'privacyStatus': 'public',
                    'selfDeclaredMadeForKids': False
                }
            }

            # MediaFileUpload with specific parameters for Shorts
            media = MediaFileUpload(
                video_path,
                resumable=True,
                chunksize=1024*1024  # 1MB chunks
            )

            # Call the API's videos.insert method to create and upload the video
            request = self.youtube.videos().insert(
                part=','.join(body.keys()),
                body=body,
                media_body=media
            )

            response = request.execute()
            logging.info(f"Short uploaded successfully! Video ID: {response['id']}")
            return response['id']

        except HttpError as error:
            logging.error(f"An error occurred: {error}")
            return None
        finally:
            media.close()

def main():
    """
    Upload all generated videos to YouTube Shorts
    """
    # Ensure you have the videos directory mounted or uploaded
    videos_dir = "/content/Video/final_videos/"

    # Create the directory if it doesn't exist
    os.makedirs(videos_dir, exist_ok=True)

    # Prompt to upload videos if directory is empty
    if not os.listdir(videos_dir):
        print("Please upload your video files to the /content/Video/final_videos/ directory.")
        uploaded_files = files.upload()

        # Move uploaded files to the correct directory
        for filename in uploaded_files:
            os.rename(filename, os.path.join(videos_dir, filename))

    # Find video files
    video_files = [
        os.path.join(videos_dir, f)
        for f in os.listdir(videos_dir)
        if f.endswith('.mp4')
    ]

    # Validate video files
    if not video_files:
        raise ValueError("No video files found to upload.")

    # Initialize uploader
    uploader = YouTubeShortsUploader()

    # Upload each video
    for video_path in video_files:
        # Extract book number from filename
        book_number = os.path.splitext(os.path.basename(video_path))[0].split('_')[-1]

        try:
            uploader.upload_short(video_path, book_number)
        except Exception as e:
            logging.error(f"Failed to upload {video_path}: {e}")

if __name__ == '__main__':
    main()